# exp009_eval
[Notion](https://www.notion.so/exp009-b202c25254da474497594650a837f62b?pvs=4)

In [1]:
import os
import random
import sys
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from typing import Tuple, Any

import albumentations as A
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.cuda.amp as amp
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# リポジトリtopに移動
while os.path.basename(os.getcwd()) != 'rsna-2023':
    os.chdir('../')
    if os.getcwd() == '/':
        raise Exception('Could not find project root directory.')

from src.volume_classification.dataset import TestDataset, load_df
from src.image_processing import windowing
from src.logger import get_logger
from src.loss import BCEDiceLoss, DiceCoef
from src.volume_classification.model import load_models
from src.volume_classification.trainer import evaluate

# Config

In [2]:
class CFG:
    exp_name = 'exp_009'
    # model config
    # timm backbone
    backbone = 'efficientnet-b1'
    n_ch = 1
    expand_ch_dim = True
    # n_class: healthy, low, high
    n_class = 3
    # hyper params
    init_lr = 1e-4
    min_lr = 1e-6
    weight_decay = 1e-4
    image_size = (128, 128, 128)
    batch_size = 64
    amp = True
    n_epoch = 20
    pretrain = False
    freeze_epochs = 0
    noaug_epochs = 1
    # fold config
    n_fold = 6
    include_evaluation = False
    train_folds = 1
    # path
    image_dir = "data/dataset002"
    model_save_dir = "outputs"
    # other config
    seed = 42
    num_workers = 0
    num_gpus = 2
    progress_bar = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# get label correspondences
organ_index_dict_inv = {
    0: 'liver',
    1: 'spleen',
    2: 'kidney',
    3: 'bowel'
}
organ_index_dict = {v: k for k, v in organ_index_dict_inv.items()}

# load dataframe
df_train = pd.read_csv('data/rsna-2023-abdominal-trauma-detection/train.csv')
df_train_image_level = pd.read_csv('data/rsna-2023-abdominal-trauma-detection/image_level_labels.csv')
df_train_series_meta = pd.read_csv('data/rsna-2023-abdominal-trauma-detection/train_series_meta.csv')

In [4]:
df = load_df(CFG)
df_eval = df[df["fold"] == 0].reset_index(drop=True)
df_eval.head()

,patient_id,series_id,organ,image_path,healthy,low,high,fold
0,10007,47578,liver,data/dataset002/10007/47578/liver.npy,1,0,0,0
1,10007,47578,spleen,data/dataset002/10007/47578/spleen.npy,1,0,0,0
2,10007,47578,kidney,data/dataset002/10007/47578/kidney.npy,1,0,0,0
3,10205,65236,liver,data/dataset002/10205/65236/liver.npy,1,0,0,0
4,10205,65236,spleen,data/dataset002/10205/65236/spleen.npy,1,0,0,0


In [5]:
def apply_preprocess(image: np.ndarray)-> np.ndarray:
    """データ前処理. カスタマイズして使用.
    Args:
        image (numpy.ndarray): HU値のCT画像.
    Returns:
        image (numpy.ndarray): windowing及び0~1に正規化.
    """
    # 0~1に正規化
    image = windowing(image, wl=0, ww=400, mode="float32")
    return image

# Load models

In [6]:
# モデルの読み込み
models = load_models(CFG, mode="final")

# Evaluation

In [13]:
def evaluate_series(CFG: Any, df: pd.DataFrame, models: list, pid: int, sid: int) -> dict:
    """患者ごと(シリーズごと)の評価を行う.
    Args:
        CFG (Any): Config
        models (list): 学習済みモデルのリスト
        pid (int): 患者ID
        sid (int): シリーズID
    Returns:
        dict: 評価結果
    """
    # 評価用データセットの作成
    df_res = df[(df["patient_id"] == pid) & (df["series_id"] == sid)].reset_index(drop=True)
    ds = TestDataset(CFG, df_res, preprocess=apply_preprocess)
    eval_iterator = DataLoader(
        ds,
        shuffle=False,
        batch_size=CFG.batch_size,
        num_workers=CFG.num_workers
    )
    # 推論
    result = evaluate(CFG, models, eval_iterator)
    return result

In [14]:
pid, sid = df_eval["patient_id"][0], df_eval["series_id"][0]
result = evaluate_series(CFG, df_eval, models, pid, sid)

In [20]:
def print_result(pid: int, sid: int, result: dict)-> None:
    """予測結果をそのまま表示."""
    print(f"pid: {pid}, sid: {sid}")
    label = result["label"]
    pred = result["pred"]
    for organ, index in organ_index_dict.items():
        if index == 3:
            continue
        label_idx = np.argmax(label[index])
        print(f"{organ}: label: {label_idx}, pred: {pred[index, label_idx]*100:.3f}%")


In [22]:
for i in range(100, 200, 3): #3: oraganごとにrowがあるため
    pid, sid = df_eval["patient_id"][i], df_eval["series_id"][i]
    result = evaluate_series(CFG, df_eval, models, pid, sid)
    print_result(pid, sid, result)

pid: 12667, sid: 16083
liver: label: 0, pred: 98.971%
spleen: label: 0, pred: 99.347%
kidney: label: 0, pred: 99.544%
pid: 12744, sid: 37732
liver: label: 0, pred: 96.048%
spleen: label: 0, pred: 99.212%
kidney: label: 0, pred: 99.319%
pid: 12744, sid: 47219
liver: label: 0, pred: 92.385%
spleen: label: 0, pred: 94.906%
kidney: label: 0, pred: 99.453%
pid: 12967, sid: 55621
liver: label: 1, pred: 11.255%
spleen: label: 0, pred: 76.674%
kidney: label: 0, pred: 99.428%
pid: 13012, sid: 38013
liver: label: 0, pred: 95.063%
spleen: label: 0, pred: 96.236%
kidney: label: 0, pred: 99.339%
pid: 13106, sid: 12049
liver: label: 0, pred: 85.639%
spleen: label: 0, pred: 95.305%
kidney: label: 0, pred: 97.400%
pid: 13106, sid: 5011
liver: label: 0, pred: 76.523%
spleen: label: 0, pred: 94.961%
kidney: label: 0, pred: 90.396%
pid: 13231, sid: 31094
liver: label: 0, pred: 99.621%
spleen: label: 0, pred: 99.707%
kidney: label: 0, pred: 99.866%
pid: 13231, sid: 769
liver: label: 0, pred: 99.851%
splee